In [6]:
import pandas as pd
import spacy

df_train= pd.read_csv("politicES_phase_2_train_public.csv", on_bad_lines='skip')
df_test= pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')

In [7]:
# Agrupar los tweets por 'label' y 'ideology_multiclass' y luego concatenar los tweets
df_train= df_train.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()
df_test= df_test.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()

In [8]:
df_train.drop('label', axis=1, inplace=True)
df_test.drop('label', axis=1, inplace=True)

In [1]:
!pip install -U spacy

In [2]:
!python -m spacy download es_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
nlp = spacy.load('es_core_news_lg')

def preprocess_text(tweet):
    # Procesa el tweet usando el modelo de spaCy
    doc = nlp(tweet)
    # Lematiza y elimina stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [9]:
df_train['clean_tweet'] = df_train['tweet'].apply(preprocess_text)
df_test['clean_tweet'] = df_test['tweet'].apply(preprocess_text)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

vectorizador = TfidfVectorizer()

X_train_tfidf = vectorizador.fit_transform(df_train['clean_tweet'])
X_test_tfidf = vectorizador.transform(df_test['clean_tweet'])

y_train = df_train['ideology_multiclass']
y_test = df_test['ideology_multiclass']

logistic_model = LogisticRegression(C=1, penalty= 'l2',solver='lbfgs')
logistic_model.fit(X_train_tfidf, y_train)

y_pred = logistic_model.predict(X_test_tfidf)


macro_f1 = f1_score(y_test, y_pred, average='macro')
print(macro_f1)
print(classification_report(y_test, y_pred))


0.5596248334084345
                precision    recall  f1-score   support

          left       0.85      0.45      0.59       117
 moderate_left       0.58      0.87      0.69       210
moderate_right       0.64      0.63      0.64       153
         right       0.87      0.19      0.32        67

      accuracy                           0.63       547
     macro avg       0.73      0.54      0.56       547
  weighted avg       0.69      0.63      0.61       547

